# 트리의 앙상블

In [ ]:
# 정형 데이터 : 어떤 구조로 되어있다는 것 - CSV나 DB혹은 Excel에 저장하기 쉽다.
# 비정형 데이터 : DB나 엑셀에 저장하기 어려운 것들 - ex) 텍스트, 사진, 음악 등등

# 정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘이 앙상블 학습. 
# - 대부분 결정트리를 기반으로 만들어져 있다.
# 비 정형 데이터는 신경망 알고리즘을 이용해 사진을 인식하고 텍스트를 이해하는 모델을
# 만들 수 있다.

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-3.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

## 랜덤포레스트
: 앙상블 학습의 대표 주자 중 하나로 안정적인 성능.
- 결정 트리를 랜덤하게 만들어 결정 트리의 숲을 만드는 것. 그리고 각 결정 트리의 예측을 사용해 최종 예측을 만든다.

- 랜덤 포레스트는 각 트리를 훈련하기 위한 데이터를 랜덤하게 만드는데, 이 데이터를 만드는 방법이 독특 : 우리가 입력한 훈련 데이터에서 랜덤하게 샘플을 추출하여 훈련 데이터를 만들고, 이때 샘플이 중복되어 추출가능.

(뽑았던 거를 다시 넣고 다시 뽑기 때문) = > 이렇게 만들어진 샘플을 부트스트랩 샘플이라 함.

- 부트스트랩 샘플은 훈련 세트의 크기와 같게 만든다.1000개 가방에서 중복하여 1000개의 샘플을 뽑기 때문에 부트스트랩 샘플은 훈련 세트와 크기가 같다.

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine-date')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [7]:
from sklearn.model_selection import cross_validate
# 교차 검증
from sklearn.ensemble import RandomForestClassifier
# 기본적으로 100개의 결정 트리를 사용 -> n_jobs 매개변수를 -1로 지정하여
# 모든 CPU 코어를 사용하는 게 좋음.


rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
# return_train_score가 True면 검증 점수 뿐만 아니라 훈련 세트에 대한 점수도 같이 반환.
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [8]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)
# 결정트리에서 만든 특성 중요도와 비교했을 때 : 각각 알코올 도수, 당도, pH였는데
# 두번째 특성인 당도의 중요도가 많이 감소하고 , 알코올 도수와 pH의 중요도가 많이 상승.
# 이런 이유는 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문
# 그 결과 하나의 특성에 과도하게 집중X, 좀 더 많은 특성이 훈련에 기여할 기회 많아져

[0.23167441 0.50039841 0.26792718]


In [9]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
# 랜덤 포레스트에는 자체적으로 모델을 평가하는 점수를 얻을 수 있다.
# 부트스트랩 샘플에 포함되지 않고 남는 샘플 -> 이런 샘플을 OOB(아웃 오브 백)샘플.
# OOB 점수 출력
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라트리
: 랜덤포레스트와 비슷하게 동작.
기본적으로 100개 결정 트리 훈련
랜덤 포레스트와 동일하게 결정 트리가 제공하는 대부분의 매개변수 지원.
또한 전체 특성 중에 일부 특성을 랜덤하게 선택하여 노드 분할하는 데 사용

랜포와 차이점은 부트스트랩 샘플 사용X

In [10]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
# 엑스트라 트리가 사용하는 spliter 매개 변수는 random으로 고정인 결정 트리.
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [11]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅 
: 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식. 

기본 - 깊이가 3인 결정 트리 100개 사용

경사 하강법을 사용하여 트리를 앙상블에 추가.

경사하강법은 손실함수를 산으로 정의하고 가장 낮은 곳을 찾아 내려오는 과정으로 설명했었고, 이때 가장 낮은 곳을 찾아 내려오는 방법은 모델의 가중치와 절편을 조금씩 바꾸는 것.

그레이디언트 부스팅은 결정 트리를 계속 추가하면서 가장 낮은 곳을 찾아 이동.

In [12]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [13]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
# 결정 트리 개수를 500개 / 러닝 레이트(학습률)를 0.2로
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [14]:
gb.fit(train_input, train_target)
print(gb.feature_importances_) # 랜포보다 일부 특성(당도)에 더 집중.

[0.15853457 0.68010884 0.1613566 ]


## 히스토그램 기반 부스팅

- 정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높은 알고리즘.

- 먼저 입력 특성을 256개 구간으로 나눈다.
 
- 따라서 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있다.

- 256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용. 따라서 입력에 누락된 특성이 있더라도 이를 따로 전처리할 필요가 없다.

- 트리의 개수를 지정하는 데 n_estimators 대신에 부스팅 반복 횟수를 지정하는 max_iter를 사용한다. 

In [15]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

C:\Users\Playdata\anaconda3\lib\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.9321723946453317 0.8801241948619236


In [18]:
from sklearn.inspection import permutation_importance
# permutation_importance 함수가 반환하는 객체는 반복하여 얻은
# 특성 중요도(importances), 평균(importances_mean),
# 표준편차(importances_std)를 담고 있다.
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state = 42, n_jobs=-1)
print(result.importances_mean)
# 테스트 세트의 결과를 보면 그레이디언트 부스팅과 비슷하게 조금 더 당도에 집중하고 있다는 것을 알 수 있다.


[0.08876275 0.23438522 0.08027708]


In [19]:
hgb.score(test_input, test_target)
# 성능 최종 확인.

0.8723076923076923

#### XGBoost

- 사이킷런 말고도 히스토그램 기반 그레이디언트 부스팅 알고리즘을 구현한 라이브러리 중 대표
- 다양한 부스팅 알고리즘 지원

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8827690284750664 0.8708899089361072


#### LightGBM

- 또 다른 히스토그램 기반 그레이디언트 부스팅 라이브러리는 마이크로소프트에서 만든 LightGBM : 빠르고 최신 기술을 많이 적용하고 있어 인기가 점점 높아지고 있다.

- 사실 사이킷런의 히스토그램 기반 그레이디언트 부스팅이 LightGBM에서 영향을 많이 받았다.


In [ ]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293


## 앙상블 학습 보고서

- 앙상블 학습은 정형 데이터에서 가장 뛰어난 성능을 내는 머신러닝 알고리즘 중 하나다. 대표적인 앙상블 학습은 다음과 같다.

### 사이킷런
- 랜덤 포레스트 : 부트스트랩 샘플 사용, 대표 앙상블 학습 알고리즘
- 엑스트라 트리 : 결정 트리의 노드를 랜덤하게 분할함
- 그레이디언트 부스팅 : 이전 트리의 손실을 보완하는 식으로 얕은 결정 트리를 연속하여 추가(학습률 매개 변수를 조정하여 모델의 복잡도를 제어할 수 있다.)
- 히스토그램 기반 그레이디언트 부스팅 : 훈련 데이터를 256개 정수 구간으로 나누어 노드 분할 속도가 매우 빠르고 높은 성능을 냄

#### 그리드 서치, 랜덤 서치를 사용한 하이퍼 파라미터 튜닝을 사용하면 최고 수준의 성능을 내는 머신러닝 모델을 얻을 수 있다.

### 그 외
- XGBoost
- LightGBM

------------------------------
지금까지는 입력과 타깃이 준비된 문제를 풀었다. 이런 머신러닝 분야를 지도학습이라고 부른다.
타깃이 없다면 어떨까?